In [ ]:
!pip install -U pymoo
!pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import keras
from keras import layers
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skopt import gp_minimize

In [ ]:
data = pd.read_csv("data/Credit.csv")
data

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,6,'critical/other existing credit',radio/tv,1169,'no known savings',>=7,4,'male single',none,...,'real estate',67,none,own,2,skilled,1,yes,yes,good
1,0<=X<200,48,'existing paid',radio/tv,5951,<100,1<=X<4,2,'female div/dep/mar',none,...,'real estate',22,none,own,1,skilled,1,none,yes,bad
2,'no checking',12,'critical/other existing credit',education,2096,<100,4<=X<7,2,'male single',none,...,'real estate',49,none,own,1,'unskilled resident',2,none,yes,good
3,<0,42,'existing paid',furniture/equipment,7882,<100,4<=X<7,2,'male single',guarantor,...,'life insurance',45,none,'for free',1,skilled,2,none,yes,good
4,<0,24,'delayed previously','new car',4870,<100,1<=X<4,3,'male single',none,...,'no known property',53,none,'for free',2,skilled,2,none,yes,bad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,'no checking',12,'existing paid',furniture/equipment,1736,<100,4<=X<7,3,'female div/dep/mar',none,...,'real estate',31,none,own,1,'unskilled resident',1,none,yes,good
996,<0,30,'existing paid','used car',3857,<100,1<=X<4,4,'male div/sep',none,...,'life insurance',40,none,own,1,'high qualif/self emp/mgmt',1,yes,yes,good
997,'no checking',12,'existing paid',radio/tv,804,<100,>=7,4,'male single',none,...,car,38,none,own,1,skilled,1,none,yes,good
998,<0,45,'existing paid',radio/tv,1845,<100,1<=X<4,4,'male single',none,...,'no known property',23,none,'for free',1,skilled,1,yes,yes,bad


In [ ]:
cat_columns = list(data.select_dtypes(include=['object']).columns)

In [ ]:
le = LabelEncoder()
for col in cat_columns:
    data[col] = le.fit_transform(data[col])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    data.iloc[:, :-1],
    data.iloc[:, -1],
    test_size=0.3,
    random_state=0
)

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(64, activation="relu", input_shape=(x_train.shape[1],)),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
history = model.fit(x_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
18/18 [==============================] - 2s 16ms/step - loss: 210.4392 - accuracy: 0.3536 - val_loss: 50.0975 - val_accuracy: 0.6643
Epoch 2/20
18/18 [==============================] - 0s 4ms/step - loss: 54.2441 - accuracy: 0.7018 - val_loss: 17.5223 - val_accuracy: 0.6643
Epoch 3/20
18/18 [==============================] - 0s 4ms/step - loss: 10.0374 - accuracy: 0.5214 - val_loss: 1.1056 - val_accuracy: 0.5857
Epoch 4/20
18/18 [==============================] - 0s 4ms/step - loss: 4.2266 - accuracy: 0.5500 - val_loss: 2.9606 - val_accuracy: 0.6643
Epoch 5/20
18/18 [==============================] - 0s 5ms/step - loss: 1.4994 - accuracy: 0.6196 - val_loss: 3.0600 - val_accuracy: 0.6643
Epoch 6/20
18/18 [==============================] - 0s 5ms/step - loss: 2.0795 - accuracy: 0.6179 - val_loss: 0.9998 - val_accuracy: 0.5786
Epoch 7/20
18/18 [==============================] - 0s 6ms/step - loss: 1.1927 - accuracy: 0.6446 - val_loss: 0.9675 - val_accuracy: 0.5643
Epoch 8/20
18

In [ ]:
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print(f'Testing accuracy: {accuracy}')

10/10 [==============================] - 0s 2ms/step
Testing accuracy: 0.35


In [ ]:
def build_model():
    # Criar o modelo da rede neural convolucional
    model = keras.Sequential([
        keras.layers.Dense(64, activation="relu", input_shape=(20,)),
        keras.layers.Dense(32, activation="relu"),
        keras.layers.Dense(1, activation="sigmoid")
    ])

    return model

In [ ]:
def chebyshev_loss(y_true, y_pred, model, lambda1, lambda2):
    # Função de perda Categorical Crossentropy
    ce_loss = keras.losses.BinaryCrossentropy()(y_true, y_pred)

    # Função de regularização L2
    l2_loss = sum([tf.nn.l2_loss(layer.kernel) for layer in model.layers if 'Conv2D' in str(layer) or 'Dense' in str(layer)])

    # Combinar as funções de perda com os pesos de Chebyshev
    combined_loss = lambda1 * ce_loss + lambda2 * l2_loss

    return combined_loss

In [ ]:
def objective(params):
    lambda1, lambda2 = params
    
    # Construir o modelo da CNN
    model = build_model()

    # Compilar o modelo com a função de perda personalizada e otimizador
    model.compile(loss=lambda y_true, y_pred: chebyshev_loss(y_true, y_pred, model, lambda1, lambda2), optimizer=keras.optimizers.Adam(), metrics=["accuracy"])

    # Treinar o modelo
    model.fit(x_train, y_train, batch_size=128, epochs=2, validation_split=0.1, verbose=1)

    # Avaliar o desempenho do modelo
    score = model.evaluate(x_test, y_test, verbose=0)
    
    return score[0]

In [ ]:
space = [(0.0001, 0.01, "uniform"), (0.0001, 0.01, "uniform")]

In [ ]:
result = gp_minimize(func=objective, dimensions=space, n_calls=10, random_state=42, verbose=1)

Iteration No: 1 started. Evaluating function at random point.
Epoch 1/2
5/5 [==============================] - 1s 69ms/step - loss: 1.2707 - accuracy: 0.3016 - val_loss: 0.5602 - val_accuracy: 0.3429
Epoch 2/2
5/5 [==============================] - 0s 11ms/step - loss: 0.2944 - accuracy: 0.4762 - val_loss: 0.3047 - val_accuracy: 0.6571
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 1.6918
Function value obtained: 0.2452
Current minimum: 0.2452
Iteration No: 2 started. Evaluating function at random point.
Epoch 1/2
5/5 [==============================] - 2s 87ms/step - loss: 1.5290 - accuracy: 0.6984 - val_loss: 1.1368 - val_accuracy: 0.6571
Epoch 2/2
5/5 [==============================] - 0s 22ms/step - loss: 0.6087 - accuracy: 0.6984 - val_loss: 0.3804 - val_accuracy: 0.3429
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 2.1046
Function value obtained: 0.3888
Current minimum: 0.2452
Iteration No: 3 started. Evaluating function at random point.


In [ ]:
best_lambda1, best_lambda2 = result.x
print(f"Melhores hiperparâmetros encontrados: lambda1 = {best_lambda1}, lambda2 = {best_lambda2}")

Melhores hiperparâmetros encontrados: lambda1 = 0.004513744253250554, lambda2 = 0.001089751666598229


In [ ]:
#Construir e treinar o modelo base
base_model = build_model()
base_model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(), metrics=["accuracy"])
base_model.fit(x_train, y_train, batch_size=128, epochs=2, validation_split=0.1, verbose=1)

Epoch 1/2
5/5 [==============================] - 2s 132ms/step - loss: 96.0279 - accuracy: 0.3016 - val_loss: 15.9695 - val_accuracy: 0.6429
Epoch 2/2
5/5 [==============================] - 0s 16ms/step - loss: 34.8505 - accuracy: 0.6984 - val_loss: 57.9209 - val_accuracy: 0.6571


In [ ]:
#Avaliar o modelo base
score = base_model.evaluate(x_test, y_test, verbose=1)
print(f"Test loss: {score[0]}, Test accuracy: {score[1]}")

10/10 [==============================] - 0s 3ms/step - loss: 43.3378 - accuracy: 0.7133
Test loss: 43.33778381347656, Test accuracy: 0.7133333086967468


In [ ]:
#Construir e treinar o melhor modelo
best_model = build_model()
best_model.compile(loss=lambda y_true, y_pred: chebyshev_loss(y_true, y_pred, best_model, best_lambda1, best_lambda2),
optimizer=keras.optimizers.Adam(), metrics=["accuracy"])
best_model.fit(x_train, y_train, batch_size=128, epochs=2, validation_split=0.1, verbose=1)

Epoch 1/2
5/5 [==============================] - 2s 89ms/step - loss: 0.2126 - accuracy: 0.6984 - val_loss: 0.1149 - val_accuracy: 0.3571
Epoch 2/2
5/5 [==============================] - 0s 15ms/step - loss: 0.1168 - accuracy: 0.3190 - val_loss: 0.0666 - val_accuracy: 0.6571


In [ ]:
#Avaliar o melhor modelo
score = best_model.evaluate(x_test, y_test, verbose=1)
print(f"Test loss: {score[0]}, Test accuracy: {score[1]}")

10/10 [==============================] - 0s 3ms/step - loss: 0.0592 - accuracy: 0.7133
Test loss: 0.0592220313847065, Test accuracy: 0.7133333086967468
